# 🎨 管理 Questbook 图标

这个 notebook 用于：
- 📤 将图片转换为 Base64
- 🔄 批量处理图标
- 📦 更新图标数据
- 🗜️ 压缩为 GTBL 格式

## 支持的图标类型
- PNG
- WebP
- JPEG/JPG

In [ ]:
import json
import gzip
import base64
from pathlib import Path
from PIL import Image
import io

print("✓ 导入模块成功")

## ⚙️ 配置

In [ ]:
# 设置版本代号
VERSION_CODE = "custom"  # 修改为你的版本代号

# 路径配置
BASE_DIR = Path("..") / "bin" / "version" / VERSION_CODE
ICONS_DIR = BASE_DIR / "quests_icons"

if not BASE_DIR.exists():
    print(f"❌ 版本目录不存在: {BASE_DIR}")
else:
    print(f"✓ 版本目录: {BASE_DIR}")
    print(f"✓ 图标目录: {ICONS_DIR}")

## 🔧 工具函数

In [ ]:
def image_to_base64(image_path, target_format='webp', quality=95):
    """
    将图片转换为 Base64
    
    参数:
        image_path: 图片路径
        target_format: 目标格式 ('webp', 'png', 'jpeg')
        quality: 压缩质量 (1-100, 仅对 webp 和 jpeg 有效)
    """
    try:
        # 打开图片
        with Image.open(image_path) as img:
            # 转换为 RGB（如果有透明通道且目标格式不支持）
            if target_format.lower() == 'jpeg' and img.mode in ('RGBA', 'LA', 'P'):
                # 创建白色背景
                background = Image.new('RGB', img.size, (255, 255, 255))
                if img.mode == 'P':
                    img = img.convert('RGBA')
                background.paste(img, mask=img.split()[-1] if img.mode == 'RGBA' else None)
                img = background
            
            # 保存到内存
            buffer = io.BytesIO()
            
            if target_format.lower() == 'webp':
                img.save(buffer, format='WebP', quality=quality, method=6)
            elif target_format.lower() == 'png':
                img.save(buffer, format='PNG', optimize=True)
            elif target_format.lower() in ['jpeg', 'jpg']:
                img.save(buffer, format='JPEG', quality=quality, optimize=True)
            else:
                raise ValueError(f"不支持的格式: {target_format}")
            
            # 转换为 Base64
            buffer.seek(0)
            base64_str = base64.b64encode(buffer.read()).decode('utf-8')
            
            return base64_str
    
    except Exception as e:
        print(f"❌ 转换失败 {image_path}: {e}")
        return None

def save_icon_data(icon_dir, icon_data, filename_prefix):
    """保存图标数据为 JSON 和 GTBL"""
    # 保存 JSON
    json_path = icon_dir / f"{filename_prefix}.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(icon_data, f, ensure_ascii=False, indent=2)
    print(f"  ✓ JSON: {json_path.relative_to(Path('..'))}")
    
    # 保存 GTBL (压缩)
    gtbl_path = icon_dir / f"{filename_prefix}.gtbl"
    with gzip.open(gtbl_path, 'wb') as f:
        f.write(json.dumps(icon_data, ensure_ascii=False).encode('utf-8'))
    print(f"  ✓ GTBL: {gtbl_path.relative_to(Path('..'))}")
    
    # 显示文件大小
    json_size = json_path.stat().st_size
    gtbl_size = gtbl_path.stat().st_size
    compression_ratio = (1 - gtbl_size / json_size) * 100
    print(f"  📊 大小: JSON {json_size} bytes, GTBL {gtbl_size} bytes (压缩率 {compression_ratio:.1f}%)")

print("✓ 工具函数定义完成")

## 📤 单个图标转换

### 方法1：从文件转换

In [ ]:
# ========== 配置 ==========
IMAGE_PATH = "path/to/your/icon.png"  # 修改为你的图标路径
TARGET_FORMAT = "webp"                 # 目标格式: 'webp', 'png', 'jpeg'
QUALITY = 95                           # 压缩质量 (1-100)

# 转换
if Path(IMAGE_PATH).exists():
    base64_str = image_to_base64(IMAGE_PATH, TARGET_FORMAT, QUALITY)
    
    if base64_str:
        print(f"✓ 转换成功！")
        print(f"  文件: {IMAGE_PATH}")
        print(f"  格式: {TARGET_FORMAT}")
        print(f"  Base64长度: {len(base64_str)} 字符")
        print(f"\n📋 Base64结果 (前100字符):")
        print(base64_str[:100] + "...")
        
        # 保存到变量供后续使用
        CONVERTED_BASE64 = base64_str
else:
    print(f"❌ 文件不存在: {IMAGE_PATH}")
    print("提示: 将图标文件放到项目目录中，然后修改 IMAGE_PATH")

## 🔄 批量转换图标

### 批量处理文件夹中的所有图标

In [ ]:
# ========== 配置 ==========
SOURCE_DIR = Path("path/to/icon/folder")  # 修改为包含图标的文件夹路径
TARGET_FORMAT = "webp"                     # 目标格式
QUALITY = 95                               # 压缩质量

# 支持的图片格式
SUPPORTED_FORMATS = ['.png', '.jpg', '.jpeg', '.webp', '.bmp']

# 批量转换
if SOURCE_DIR.exists():
    icon_files = []
    for ext in SUPPORTED_FORMATS:
        icon_files.extend(SOURCE_DIR.glob(f"*{ext}"))
    
    if icon_files:
        print(f"找到 {len(icon_files)} 个图标文件\n")
        
        converted_icons = {}
        
        for icon_path in icon_files:
            # 使用文件名（不含扩展名）作为 key
            icon_name = icon_path.stem
            
            print(f"转换: {icon_path.name} ... ", end="")
            base64_str = image_to_base64(icon_path, TARGET_FORMAT, QUALITY)
            
            if base64_str:
                converted_icons[icon_name] = base64_str
                print(f"✓ ({len(base64_str)} chars)")
            else:
                print("❌")
        
        print(f"\n✓ 批量转换完成！")
        print(f"  成功: {len(converted_icons)}/{len(icon_files)}")
        
        # 保存到变量供后续使用
        BATCH_CONVERTED_ICONS = converted_icons
    else:
        print(f"❌ 文件夹中没有找到图标文件: {SOURCE_DIR}")
else:
    print(f"❌ 文件夹不存在: {SOURCE_DIR}")
    print("提示: 创建文件夹并将图标放入其中")

## 📝 更新任务图标

### 替换单个任务的图标

In [ ]:
# ========== 配置 ==========
QUESTLINE_NAME = "WelcomeQuests"  # 任务线名称
QUEST_ID = "welcome_001"          # 任务ID
NEW_ICON_BASE64 = "your_base64_string_here"  # 新的图标 Base64（从上面的转换结果复制）

# 如果上面转换了图标，可以直接使用
# NEW_ICON_BASE64 = CONVERTED_BASE64

# 更新图标
icon_dir = ICONS_DIR / "QuestIcon" / QUESTLINE_NAME
icon_json_path = icon_dir / f"{QUESTLINE_NAME}.json"

if icon_json_path.exists():
    # 加载现有数据
    with open(icon_json_path, 'r', encoding='utf-8') as f:
        icon_data = json.load(f)
    
    # 更新图标
    if QUEST_ID in icon_data:
        old_size = len(icon_data[QUEST_ID])
        icon_data[QUEST_ID] = NEW_ICON_BASE64
        new_size = len(NEW_ICON_BASE64)
        
        print(f"✓ 图标已更新")
        print(f"  任务ID: {QUEST_ID}")
        print(f"  旧大小: {old_size} 字符")
        print(f"  新大小: {new_size} 字符")
        print(f"  变化: {'+' if new_size > old_size else ''}{new_size - old_size} 字符")
    else:
        # 添加新图标
        icon_data[QUEST_ID] = NEW_ICON_BASE64
        print(f"✓ 添加新图标: {QUEST_ID}")
    
    # 保存
    print(f"\n保存图标数据...")
    save_icon_data(icon_dir, icon_data, QUESTLINE_NAME)
    print(f"\n✓ 图标更新完成！")
else:
    print(f"❌ 图标文件不存在: {icon_json_path}")

## 📦 批量更新任务图标

In [ ]:
# ========== 配置 ==========
QUESTLINE_NAME = "WelcomeQuests"  # 任务线名称

# 使用批量转换的结果
# 假设图标文件名就是任务ID
if 'BATCH_CONVERTED_ICONS' in globals():
    icon_dir = ICONS_DIR / "QuestIcon" / QUESTLINE_NAME
    icon_json_path = icon_dir / f"{QUESTLINE_NAME}.json"
    
    if icon_json_path.exists():
        # 加载现有数据
        with open(icon_json_path, 'r', encoding='utf-8') as f:
            icon_data = json.load(f)
        
        # 批量更新
        updated = 0
        added = 0
        
        for quest_id, base64_str in BATCH_CONVERTED_ICONS.items():
            if quest_id in icon_data:
                icon_data[quest_id] = base64_str
                updated += 1
            else:
                icon_data[quest_id] = base64_str
                added += 1
        
        print(f"✓ 批量更新完成")
        print(f"  更新: {updated} 个")
        print(f"  新增: {added} 个")
        print(f"  总计: {len(icon_data)} 个图标")
        
        # 保存
        print(f"\n保存图标数据...")
        save_icon_data(icon_dir, icon_data, QUESTLINE_NAME)
        print(f"\n✓ 批量更新完成！")
    else:
        print(f"❌ 图标文件不存在: {icon_json_path}")
else:
    print("❌ 没有批量转换结果")
    print("提示: 先运行上面的'批量转换图标'单元格")

## 🎨 更新任务线图标

In [ ]:
# ========== 配置 ==========
QUESTLINE_NAME = "WelcomeQuests"  # 任务线名称
NEW_LINE_ICON_BASE64 = "your_base64_string_here"  # 新的任务线图标 Base64

# 或使用转换结果
# NEW_LINE_ICON_BASE64 = CONVERTED_BASE64

# 更新任务线图标
line_icon_dir = ICONS_DIR / "QuestLineIcon"
line_icon_json_path = line_icon_dir / f"{QUESTLINE_NAME}.json"

if line_icon_json_path.exists():
    # 加载数据
    with open(line_icon_json_path, 'r', encoding='utf-8') as f:
        line_icon_data = json.load(f)
    
    # 更新
    old_size = len(line_icon_data.get(QUESTLINE_NAME, ""))
    line_icon_data[QUESTLINE_NAME] = NEW_LINE_ICON_BASE64
    new_size = len(NEW_LINE_ICON_BASE64)
    
    print(f"✓ 任务线图标已更新")
    print(f"  任务线: {QUESTLINE_NAME}")
    print(f"  新大小: {new_size} 字符")
    
    # 保存
    print(f"\n保存图标数据...")
    save_icon_data(line_icon_dir, line_icon_data, QUESTLINE_NAME)
    print(f"\n✓ 任务线图标更新完成！")
else:
    print(f"❌ 图标文件不存在: {line_icon_json_path}")

## 📊 图标统计

In [ ]:
print("="*60)
print("📊 图标统计")
print("="*60)

# 任务线图标
line_icon_dir = ICONS_DIR / "QuestLineIcon"
line_icon_count = 0
line_icon_size = 0

for json_file in line_icon_dir.glob("*.json"):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        line_icon_count += len(data)
    line_icon_size += json_file.stat().st_size

print(f"\n任务线图标:")
print(f"  数量: {line_icon_count}")
print(f"  JSON大小: {line_icon_size:,} bytes")

# 任务图标
quest_icon_dir = ICONS_DIR / "QuestIcon"
total_quest_icons = 0
total_json_size = 0
total_gtbl_size = 0

for questline_dir in quest_icon_dir.iterdir():
    if questline_dir.is_dir():
        for json_file in questline_dir.glob("*.json"):
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                icon_count = len(data)
                total_quest_icons += icon_count
            
            json_size = json_file.stat().st_size
            total_json_size += json_size
            
            gtbl_file = json_file.with_suffix('.gtbl')
            if gtbl_file.exists():
                gtbl_size = gtbl_file.stat().st_size
                total_gtbl_size += gtbl_size
            
            print(f"  {questline_dir.name}: {icon_count} 个图标")

print(f"\n任务图标总计:")
print(f"  数量: {total_quest_icons}")
print(f"  JSON大小: {total_json_size:,} bytes")
print(f"  GTBL大小: {total_gtbl_size:,} bytes")
if total_json_size > 0:
    compression_ratio = (1 - total_gtbl_size / total_json_size) * 100
    print(f"  压缩率: {compression_ratio:.1f}%")

print("\n" + "="*60)

## 💡 使用技巧

### 推荐图标规格
- **尺寸**: 64x64 像素（或 32x32, 128x128）
- **格式**: WebP（最佳压缩）或 PNG（更好兼容性）
- **质量**: WebP 使用 90-95 质量

### 批量处理工作流
1. 将所有图标放在一个文件夹
2. 文件名设为任务ID（如 `welcome_001.png`）
3. 运行批量转换
4. 批量更新到questbook

### 图标优化建议
- 使用 WebP 格式可减少 20-50% 文件大小
- 简单图标可以降低质量到 80-85
- 避免使用过大的图标（建议 < 50KB）